In [1]:
#Using the packages for installing Vowpal Wabbit
!pip install boost
!pip install libboost-program-options-dev zlib1g-dev libboost-python-dev -y
!python -m pip install vowpalwabbit


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [7]:
!pip freeze

appnope==0.1.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
attrs==21.4.0
backcall==0.2.0
black==21.12b0
bleach==4.1.0
blurhash==1.1.4
boost==0.1
brotlipy @ file:///Users/runner/miniforge3/conda-bld/brotlipy_1636012319596/work
caer==2.0.8
certifi==2021.10.8
cffi @ file:///Users/runner/miniforge3/conda-bld/cffi_1636046173594/work
charset-normalizer @ file:///home/conda/feedstock_root/build_artifacts/charset-normalizer_1638815705608/work
click==8.0.3
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1602866480661/work
conda==4.11.0
conda-package-handling @ file:///Users/runner/miniforge3/conda-bld/conda-package-handling_1636022092101/work
cryptography @ file:///Users/runner/miniforge3/conda-bld/cryptography_1637648810489/work
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.3
executing==0.8.2
idna @ file:///home/conda/feedstock_root/build_artifacts/idna_1609836280497/work
ipykernel==6.7.0
ipython==8.0.0

ipywidgets==7.6.5
jedi==0.1

In [8]:
!python3 -m pip install vowpalwabbit
import vowpalwabbit

ModuleNotFoundError: No module named 'vowpalwabbit'

In [2]:
#Importing the required libraries
import pandas as pd
from vowpalwabbit import pyvw 
import numpy as np

ModuleNotFoundError: No module named 'vowpalwabbit'

In [ ]:
#Uploading the 1000 Bucket size CSVs of ACKTR and DQN
# from google.colab import files
# files.upload()

In [ ]:
#Creating a dataframe
vac_df = pd.read_csv('DQN_1KBuc_CB_Input_dat.csv')
#vac_df = pd.read_csv('ACKTR_1KBuc_CB_Input_dat.csv')
vac_df = vac_df[["Date","State...UT","Confirmed.cases","Death.Rate","Recovery.Rate","Population","Susceptible","Overall.Hospital.Beds","Overall.ICU.Beds","Ventilators","Age.Distribution","Action","Reward","Recovered.Cases","Deceased.cases","Percentage.distribution"]]
#vac_df = pd.read_csv('DQN_1KBuc_CB_Input_dat.csv')
vac_df=vac_df[30:]
vac_df

In [ ]:
from vowpalwabbit import pyvw 
import numpy as np

In [ ]:
#Given that our models learn well for bucket size 200-500, we consider the buckets 200, 300, 400 and 500
for var in range(200,501,100): 
    print("BUCKET SIZE",var)
    vw = pyvw.vw("--cb "+str(var))
    ### TRAINING ###
    actions_train=[]
    t=2 # 10000 initially
    while(t):
        t-=1
        for i in range(0,130,5):   #Training on 1st to 26th Dec
            arr=vac_df[i:i+5].to_numpy()    #i to i+5 for 5 states on single date
            total_confirmed=sum(arr[:,2])
            total_population=sum(arr[:,5])
            total_susceptible=sum(arr[:,6])
            total_beds=sum(arr[:,7])
            total_icu=sum(arr[:,8])
            total_venti=sum(arr[:,9])
            total_old=sum(arr[:,10])
            for s in range(0,5):          # s for 5 diff models for 5 diff states
                confirmed_ratio=arr[s][2]#/total_confirmed
                dr=arr[s][3]#/100; 
                rr=arr[s][4]#/100
                population_ratio=arr[s][5]#/total_population
                susceptible_ratio = arr[s][6]#/total_susceptible
                bed_ratio=arr[s][7]#/total_beds
                icu_ratio=arr[s][8]#/total_icu
                venti_ratio=arr[s][9]#/total_venti
                old_ratio=arr[s][10]#/total_old
                action=int(arr[s][11])
                reward=int(arr[s][12])
              
                new_action = int(abs((10*action*var + 5*var -5000)/10000))    # only for original action space of 1000 actions
                actions_train.append(new_action) 
                learn_example=str(new_action)+':'+str(-reward)+':'+str(0.9)+' | '+str(susceptible_ratio)+' '+str(confirmed_ratio)+' '+str(population_ratio)+' '+str(dr)+' '+str(rr)+' '+str(bed_ratio)+' '+str(icu_ratio)+' '+str(venti_ratio)+' '+str(old_ratio)
                #print(learn_example)
                vw.learn(learn_example)
    actions_train_unique=np.unique(actions_train)
    #print("unique action in training",actions_train_unique)
    ### TESTING ###
    actions_test = []
    for i in range(130,310,5):   # Testing on 27th Dec to 31st Jan
        arr=vac_df[i:i+5].to_numpy()    #i to i+5 for 5 states on single date
        total_confirmed=sum(arr[:,2])
        total_population=sum(arr[:,5])
        total_susceptible=sum(arr[:,6])
        total_beds=sum(arr[:,7])
        total_icu=sum(arr[:,8])
        total_venti=sum(arr[:,9])
        total_old=sum(arr[:,10])
        for s in range(5):          # s for 5 diff models for 5 diff states
            confirmed_ratio=arr[s][2]#/total_confirmed
            dr=arr[s][3]#/100
            rr=arr[s][4]#/100
            population_ratio=arr[s][5]#/total_population
            susceptible_ratio = arr[s][6]#/total_susceptible
            bed_ratio=arr[s][7]#/total_beds
            icu_ratio=arr[s][8]#/total_icu
            venti_ratio=arr[s][9]#/total_venti
            old_ratio=arr[s][10]#/total_old
            #action=int(arr[s][11])
            #reward=arr[s][12]
            test_example='| '+str(susceptible_ratio)+' '+str(confirmed_ratio)+' '+str(population_ratio)+' '+str(dr)+' '+str(rr)+' '+str(bed_ratio)+' '+str(icu_ratio)+' '+str(venti_ratio)+' '+str(old_ratio)
            a=vw.predict(test_example)
            #print(test_example)
            actions_test.append(a)
            print(a)
    actions_test_unique=np.unique(actions_test)
    #print("unique actions in testing",actions_test_unique)